In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from matplotlib import pyplot as plt
import sys
import time

import random

sys.path.append('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/')

from scripts.utils import create_one_hot_encoding

pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings("ignore")

In [2]:
random.seed(42)
customers_list = pd.read_parquet('../data/compressed_dataset/customers.parquet')[['customer_id']]['customer_id'].tolist()
random.shuffle(customers_list)

In [3]:
# test_4
final_top_k = {
    'ARulesRecommender': 12,
    'GruRecommender': 12,
    'ItemsPurchasedTogetherRecommender': 12,
    'KaggleCustomerAgeRecommender': 11,
    'KaggleExponentialDecayRecommender': 12,
    'KaggleTrendingRecommender': 12,
    'KMeansRecommender': 10,
    'LastPurchasesPopularity': 12,
    'LightFMRecommender': 12,
    'PopularByGroupsRecommender': 5,
    'SVDRecommender': 12
}

In [4]:
# # test_3
# final_top_k = {
#     'ARulesRecommender': 20,
#     'GruRecommender': 20,
#     'ItemsPurchasedTogetherRecommender': 12,
#     'KaggleCustomerAgeRecommender': 11,
#     'KaggleExponentialDecayRecommender': 12,
#     'KaggleTrendingRecommender': 12,
#     'KMeansRecommender': 10,
#     'LastPurchasesPopularity': 12,
#     'LightFMRecommender': 12,
#     'PopularByGroupsRecommender': 5,
#     'SVDRecommender': 12
# }

In [5]:
# # test_2
# final_top_k = {
#     'BiVAERecommender': 2,
# #     'LastPurchasesImageSimilarity': 2,
#     'ARulesRecommender': 8,
#     'GruRecommender': 8,
#     'ItemsPurchasedTogetherRecommender': 24,
#     'KaggleCustomerAgeRecommender': 24,
#     'KaggleExponentialDecayRecommender': 24,
#     'KaggleTrendingRecommender': 24,
#     'KMeansRecommender': 12,
#     'LastPurchasesPopularity': 20,
#     'LightFMRecommender': 10,
#     'PopularByGroupsRecommender': 6,
#     'SVDRecommender': 10
# }

In [6]:
path_to_model_preds = '/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_set'
path_to_dest = '/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/train_set_and_labels/test_4/'
num_train_weeks = 10

for i in range(num_train_weeks-1, 0, -1):
    print(i)
    for model_i, model_name in enumerate(final_top_k):
                
        top_k = final_top_k[model_name]
        
        path_to_model_pred = os.path.join(path_to_model_preds, model_name,
                                          '{}_{}.parquet'.format(model_name, i))
        
        model_pred = pd.read_parquet(path_to_model_pred)
        
        model_pred = model_pred[['customer_id', 'article_id', 'weeks_before_sub', 'score']]
        
        model_pred['customer_id'] = model_pred['customer_id'].astype(np.int32)
        model_pred['article_id'] = model_pred['article_id'].astype(np.int32)
        model_pred['weeks_before_sub'] = model_pred['weeks_before_sub'].astype(np.int32)
        
        model_pred = model_pred.sort_values('score', ascending=False)
        model_pred = model_pred.groupby(['customer_id', 'article_id']).first().reset_index()
        model_pred = model_pred.sort_values('score', ascending=False)
        model_pred = model_pred.groupby(['customer_id']).head(top_k).reset_index(drop=True)
        
#         if model_name == 'SVDRecommender':
#             model_pred = model_pred[model_pred.score != 0]
        
        model_pred.rename(columns={'score': 'score_{}'.format(model_name)}, inplace=True)

        if model_i == 0:
            models_preds = model_pred

        else:
            models_preds = models_preds.merge(model_pred, how='outer', on=['customer_id', 'article_id', 'weeks_before_sub'])

    del model_pred
        
    ### ARTICLES STATIC FEATURES
    
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/static/colour.parquet')
    models_preds = models_preds.merge(f, on=['article_id'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/static/product.parquet')
    models_preds = models_preds.merge(f, on=['article_id'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/static/product_name.parquet')
    models_preds = models_preds.merge(f, on=['article_id'], how='left')
    
    ### ARTICLES DYNAMIC FEATURES
    
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/article__frequency_purchases.parquet')
    models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/mean_sales_channel_id.parquet')
    models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_days_from_purchases.parquet')
    models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_purchases__ratios.parquet')
    models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/price.parquet')
    models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
    
    
    ### CUSTOMERS STATIC FEATURES
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/static/age.parquet')
    models_preds = models_preds.merge(f, on=['customer_id'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/static/member_status__fn.parquet')
    models_preds = models_preds.merge(f, on=['customer_id'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/static/sex_children_sport_type_purchases.parquet')
    models_preds = models_preds.merge(f, on=['customer_id'], how='left')
    
    
    ### CUSTOMERS DYNAMIC FEATURES
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/mean_price.parquet')
    models_preds = models_preds.merge(f, on=['customer_id', 'weeks_before_sub'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/mean_sales_channel_id.parquet')
    models_preds = models_preds.merge(f, on=['customer_id', 'weeks_before_sub'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/num_days_from_purchases.parquet')
    models_preds = models_preds.merge(f, on=['customer_id', 'weeks_before_sub'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/num_purchases__ratios.parquet')
    models_preds = models_preds.merge(f, on=['customer_id', 'weeks_before_sub'], how='left')
    
    ### CUSTOMERS_ARTICLES DYNAMIC FEATURES
    
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers_articles/dynamic/num_days_from_last_purchase/num_days_from_purchases_{}.parquet'.format(i))
    models_preds = models_preds.merge(f, on=['customer_id', 'article_id', 'weeks_before_sub'], how='left')
    f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers_articles/dynamic/num_purchases/num_purchases_{}.parquet'.format(i))
    models_preds = models_preds.merge(f, on=['customer_id', 'article_id', 'weeks_before_sub'], how='left')

    
    ### LABELS
    labels = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_labels/labels_{}.parquet'.format(i))
    
    d = dict.fromkeys(labels.select_dtypes(np.int64).columns, np.int32)
    labels = labels.astype(d)
    d = dict.fromkeys(labels.select_dtypes(np.float64).columns, np.float32)
    labels = labels.astype(d)
    
    labels.loc[labels.label >= 1, 'label'] = 1
    
    models_preds = models_preds.merge(labels, on=['customer_id', 'article_id', 'weeks_before_sub'], how='left')
    models_preds['label'].fillna(0, inplace=True)
    models_preds['label'] = models_preds['label'].astype(np.uint8)
    
    d = dict.fromkeys(models_preds.select_dtypes(np.int64).columns, np.int32)
    models_preds = models_preds.astype(d)
    d = dict.fromkeys(models_preds.select_dtypes(np.float64).columns, np.float32)
    models_preds = models_preds.astype(d)
    
    
    models_preds.to_parquet(os.path.join(path_to_dest, 'week_{}.parquet'.format(i)),
                           index=False)

9
8
7
6
5
4
3
2
1


In [4]:
del models_preds

NameError: name 'models_preds' is not defined

In [4]:
path_to_model_preds = '/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_set'
path_to_dest = '/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/train_set_and_labels/test_4/'
num_train_weeks = 1

num_partitions = 10

for part_i in range(10):
    
    cur_customers = customers_list[int(part_i*len(customers_list)/10) : int((part_i+1)*len(customers_list)/10)]

    for i in range(num_train_weeks-1, -1, -1):
        print(i)
        for model_i, model_name in enumerate(final_top_k):

            top_k = final_top_k[model_name]

            path_to_model_pred = os.path.join(path_to_model_preds, model_name,
                                              '{}_{}.parquet'.format(model_name, i))

            model_pred = pd.read_parquet(path_to_model_pred)
            
            model_pred = model_pred[model_pred['customer_id'].isin(cur_customers)]
            
            model_pred = model_pred[['customer_id', 'article_id', 'weeks_before_sub', 'score']]

            model_pred['customer_id'] = model_pred['customer_id'].astype(np.int32)
            model_pred['article_id'] = model_pred['article_id'].astype(np.int32)
            model_pred['weeks_before_sub'] = model_pred['weeks_before_sub'].astype(np.int32)

            model_pred = model_pred.sort_values('score', ascending=False)
            model_pred = model_pred.groupby(['customer_id', 'article_id']).first().reset_index()
            model_pred = model_pred.sort_values('score', ascending=False)
            model_pred = model_pred.groupby(['customer_id']).head(top_k).reset_index(drop=True)

    #         if model_name == 'SVDRecommender':
    #             model_pred = model_pred[model_pred.score != 0]

            model_pred.rename(columns={'score': 'score_{}'.format(model_name)}, inplace=True)

            if model_i == 0:
                models_preds = model_pred

            else:
                models_preds = models_preds.merge(model_pred, how='outer', on=['customer_id', 'article_id', 'weeks_before_sub'])

        del model_pred

        ### ARTICLES STATIC FEATURES

        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/static/colour.parquet')
        models_preds = models_preds.merge(f, on=['article_id'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/static/product.parquet')
        models_preds = models_preds.merge(f, on=['article_id'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/static/product_name.parquet')
        models_preds = models_preds.merge(f, on=['article_id'], how='left')

        ### ARTICLES DYNAMIC FEATURES

        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/article__frequency_purchases.parquet')
        models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/mean_sales_channel_id.parquet')
        models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_days_from_purchases.parquet')
        models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/num_purchases__ratios.parquet')
        models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/articles/dynamic/price.parquet')
        models_preds = models_preds.merge(f, on=['article_id', 'weeks_before_sub'], how='left')


        ### CUSTOMERS STATIC FEATURES
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/static/age.parquet')
        models_preds = models_preds.merge(f, on=['customer_id'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/static/member_status__fn.parquet')
        models_preds = models_preds.merge(f, on=['customer_id'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/static/sex_children_sport_type_purchases.parquet')
        models_preds = models_preds.merge(f, on=['customer_id'], how='left')


        ### CUSTOMERS DYNAMIC FEATURES
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/mean_price.parquet')
        models_preds = models_preds.merge(f, on=['customer_id', 'weeks_before_sub'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/mean_sales_channel_id.parquet')
        models_preds = models_preds.merge(f, on=['customer_id', 'weeks_before_sub'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/num_days_from_purchases.parquet')
        models_preds = models_preds.merge(f, on=['customer_id', 'weeks_before_sub'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers/dynamic/num_purchases__ratios.parquet')
        models_preds = models_preds.merge(f, on=['customer_id', 'weeks_before_sub'], how='left')

        ### CUSTOMERS_ARTICLES DYNAMIC FEATURES

        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers_articles/dynamic/num_days_from_last_purchase/num_days_from_purchases_{}.parquet'.format(i))
        models_preds = models_preds.merge(f, on=['customer_id', 'article_id', 'weeks_before_sub'], how='left')
        f = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/features/customers_articles/dynamic/num_purchases/num_purchases_{}.parquet'.format(i))
        models_preds = models_preds.merge(f, on=['customer_id', 'article_id', 'weeks_before_sub'], how='left')


        ### LABELS
    #     labels = pd.read_parquet('/home/juravlik/PycharmProjects/kaggle_hnm_recsys/data/ranker_train_labels/labels_{}.parquet'.format(i))

    #     d = dict.fromkeys(labels.select_dtypes(np.int64).columns, np.int32)
    #     labels = labels.astype(d)
    #     d = dict.fromkeys(labels.select_dtypes(np.float64).columns, np.float32)
    #     labels = labels.astype(d)

    #     labels.loc[labels.label >= 1, 'label'] = 1

    #     models_preds = models_preds.merge(labels, on=['customer_id', 'article_id', 'weeks_before_sub'], how='left')
    #     models_preds['label'].fillna(0, inplace=True)
    #     models_preds['label'] = models_preds['label'].astype(np.uint8)

        d = dict.fromkeys(models_preds.select_dtypes(np.int64).columns, np.int32)
        models_preds = models_preds.astype(d)
        d = dict.fromkeys(models_preds.select_dtypes(np.float64).columns, np.float32)
        models_preds = models_preds.astype(d)


        models_preds.to_parquet(os.path.join(path_to_dest, 'week_{}__part_{}.parquet'.format(i, part_i)),
                               index=False)

0
0
0
0
0
0
0
0
0
0
